<div class = "alert alert-block alert-info">
    
__Регрессия__
    
В общем то как я и говорил делаешь то же самое что и для тестового задания: регрессию по концентрации для каждой из солей. Любым методом, какой больше нравится / лучше работает.
Метрики те же : MAE, RMSE, $R^2$

Единственное что для нормальной оценки, нужно делать проверку не на отложенной выборке с помощью `train_test_split`, а по кросс-валидации. 
    
А именно, для поиска гиперпараметров есть функция `GridSearchCV`. Документация у `sklearn` довольно понятная, а потому искренне советую туда заглянуть.
</div>

In [1]:
from sklearn.model_selection import GridSearchCV

<div class = "alert alert-block alert-info">
    
__Классификация__
   
Сначала нужно привести эту задачу к задаче классификации: ввести пороговое значение, все концентрации что ниже порога получают метку "0", все что выше - метку "1".
    
А дальше подбор параметров на кросс-валидации, со стратификацией по таргету, так же чтобы количество единичек и нулей на тесте и трейне было одинаковым
    
Метрики качества : precision, recall, f1-score, ROC-AUC (последнее использовать только если у тебя порог по концетрации не такой, что слишком мало или слишком много какого то из классов (из-за большого дисбаланса классов метрика ведет себя не очень адекватно)). Лучше всего первые три показать после подбора гиперпараметров с помощью `classification_report`:
</div>

__Пример__

In [5]:
from sklearn.datasets import make_classification
from sklearn.metrics import classification_report, roc_auc_score
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split

In [20]:
# Эта штука генерирует набор объектов и меток
# Типа "модельная" классификация
X, y = make_classification(n_samples=1000)
Train_X, Test_X, Train_y, Test_y = train_test_split(X, y, test_size=0.2, stratify=y)

In [17]:
clf = LogisticRegression()
clf.fit(Train_X, Train_y)

# Обрати внимание что есть "probs" а есть "preds"
# "probs" - это вероятности принадлежности классу
# "preds" - непосредственно метки класса
# У тебя "probs" - это значения функции сигмоиды от выхода PLS модели (для мочи)
# а "preds" - это то что после отсечения по threshold

probs = clf.predict_proba(Test_X)[:,1]
preds = clf.predict(Test_X)

In [19]:
# Также обрати внимание что в roc_auc_score передается именно вероятности probs
print(f"ROC-AUC: {roc_auc_score(Test_y, probs):.3f}")
print(classification_report(Test_y, preds))

ROC-AUC: 0.965
              precision    recall  f1-score   support

           0       0.88      0.96      0.92       102
           1       0.96      0.87      0.91        98

    accuracy                           0.92       200
   macro avg       0.92      0.91      0.91       200
weighted avg       0.92      0.92      0.91       200



<div class = "alert alert-block alert-info">
    
Делаешь то же самое только на кросс-валидации
    
</div>

<div class = "alert alert-block alert-info">
    
Для адекватного использования этого вместе с `GridSearchCV` есть штука `pipeline`. Также рекомендую посмотреть примеры использования в документации
    
</div>

In [2]:
from sklearn.pipeline import make_pipeline